In [1]:
import duckdb

# Load SQL extension
%load_ext sql

# Initialize 🦆 DuckDB connection
conn = duckdb.connect()

# Import database
%sql conn --alias duckdb
%sql IMPORT DATABASE '../../data/nps';

There's a new jupysql version available (0.11.0), you're running 0.10.10. To upgrade: pip install jupysql --upgrade
Deploy Shiny apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


Loading configurations from /workspaces/fundamentals-of-data-transformation-4554074/pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True
displaylimit,10
displaycon,False


,Count
0,224


In [2]:
%%sql
SELECT
    COUNT(*) as num_parks
FROM nps_public_data.parks p
WHERE designation = 'National Park'

,num_parks
0,51


We can also count by state:

In [3]:
%%sql
SELECT
    states,
    COUNT(*) as num_parks
FROM nps_public_data.parks p
WHERE designation = 'National Park'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 20;

,states,num_parks
0,UT,5
1,CA,5
2,AZ,3
3,WA,3
4,FL,3
5,CO,3
6,SD,2
7,NM,2
8,HI,2
9,TX,2


In [4]:
%%sql
SELECT
    fullname,
    UNNEST(
        SPLIT(states, ',')::string[]
        ) as state
FROM nps_public_data.parks p
WHERE designation = 'National Park'
LIMIT 5

,fullName,state
0,Isle Royale National Park,MI
1,Black Canyon Of The Gunnison National Park,CO
2,Grand Canyon National Park,AZ
3,Crater Lake National Park,OR
4,Yellowstone National Park,ID


This _will_ produce multiple records for the same park (those in multiple states), but that's what we want! Now we can count the number of parks across state lines using an aggregation

In [5]:
%%sql
WITH park_list AS (
    SELECT
        fullname,
        UNNEST(
            SPLIT(states, ',')::string[]
            ) as state
    FROM nps_public_data.parks p
    WHERE designation = 'National Park'
)
SELECT
    state,
    COUNT(*) as num_parks
FROM park_list
GROUP BY 1
ORDER BY 2 DESC, 1
LIMIT 10

,state,num_parks
0,CA,6
1,UT,5
2,AZ,3
3,CO,3
4,FL,3
5,WA,3
6,AK,2
7,HI,2
8,MT,2
9,NM,2


To clarify, here's a common approach: to calculate the values we needed, it was necessary to structure the data as one row per state for each park. The original query provided a single row for each park with multiple states, so we had to reshape the data as a preliminary step toward achieving our final result.

This kind of intermediate data transformation is frequently required when preparing data for aggregation:

Analyze the data and decide on the type of aggregation, for example, counting parks by state.
Assess the current structure or format of the data (in this case, at the state level).
Determine the desired structure (e.g., state-park level).
Perform the necessary transformation to align the data with the desired structure, then proceed with the aggregation.
How can we identify the campgrounds with the fewest and most campsites using aggregation?

First, calculate the total number of campsites for each campground (equivalent to a national park).
Then, identify the minimum and maximum campsite counts.
Finally, filter the results based on these values.

In [6]:
%%sql
WITH park_campgrounds AS (
    SELECT
        c.name as campgroud_name,
        p.fullname as park_name,
        c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable as total_sites,
    FROM nps_public_data.campgrounds c
    INNER JOIN nps_public_data.parks p
        ON c.parkcode = p.parkcode
        AND p.designation = 'National Park'
    GROUP BY 1,2,3
), min_max_sites AS (
SELECT
    MIN(total_sites) as min_sites,
    MAX(total_sites) as max_sites
FROM park_campgrounds
WHERE total_sites > 0
)
SELECT
    campgroud_name,
    total_sites as num_sites,
    CASE total_sites WHEN min_sites THEN 'least' ELSE 'most' END as sites_rank
FROM park_campgrounds pc
INNER JOIN min_max_sites mms
    ON (pc.total_sites = mms.min_sites OR pc.total_sites = mms.max_sites)
ORDER BY num_sites, campgroud_name

,campgroud_name,num_sites,sites_rank
0,Dunbar Group Site,1,least
1,Frijole Horse Corral Campground,1,least
2,Group Campsite,1,least
3,Hay Bay Campground,1,least
4,Pickerel Cove Campground,1,least
5,Roundup Group Horse Camp,1,least
6,Upper Goodell Creek Group Campsite,1,least
7,Morefield Campground,534,most


What about the parks? We can follow similar logic

In [7]:
%%sql
WITH park_campgrounds AS (
    SELECT
        c.name as campgroud_name,
        p.fullname as park_name,
        c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable as total_sites,
    FROM nps_public_data.campgrounds c
    INNER JOIN nps_public_data.parks p
        ON c.parkcode = p.parkcode
        AND p.designation = 'National Park'
    GROUP BY 1,2,3
), park_sites AS (
    SELECT
        park_name,
        SUM(total_sites) as num_sites
    FROM park_campgrounds
    GROUP BY 1
    ORDER BY 2 DESC
), min_max_sites AS (
    SELECT
        MIN(num_sites) as min_sites,
        MAX(num_sites) as max_sites
    FROM park_sites ps
)
SELECT
    ps.*,
    CASE num_sites WHEN min_sites THEN 'least' ELSE 'most' END as sites_rank
FROM park_sites ps
INNER JOIN min_max_sites mms
    ON (num_sites = mms.min_sites or num_sites = mms.max_sites)
ORDER BY ps.num_sites DESC

,park_name,num_sites,sites_rank
0,Yellowstone National Park,2149.0,most
1,Dry Tortugas National Park,8.0,least


Another important aggregation pattern is _using boolean logic within aggregations_ as filters! For example

- `COUNT DISTINCT`
- `COUNT CASE WHEN`

This lets us filter our count _without_ filtering the underlying data.

In [9]:
%%sql
SELECT
    p.fullname as park_name,
    -- COUNT the number of campgrounds
    COUNT(DISTINCT c.name) as num_campgrounds,
    -- Get the average number of sites— what is this returning?
    AVG(numberofsitesreservable) as avg_sites_reservable,
    AVG(numberofsitesfirstcomefirstserve) as avg_sites_fcfs,
    AVG(numberofsitesreservable + numberofsitesfirstcomefirstserve) as avg_total_sites
FROM nps_public_data.campgrounds c
INNER JOIN nps_public_data.parks p
    ON c.parkcode = p.parkcode
    AND p.designation = 'National Park'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
-- Read more about aggregates here: https://duckdb.org/docs/sql/aggregates

,park_name,num_campgrounds,avg_sites_reservable,avg_sites_fcfs,avg_total_sites
0,Isle Royale National Park,36,1.194444,5.583333,6.777778
1,Yosemite National Park,14,75.642857,21.642857,97.285714
2,Glacier National Park,13,64.384615,13.307692,77.692308
3,Great Smoky Mountains National Park,13,72.538462,0.000000,72.538462
4,Guadalupe Mountains National Park,13,8.461538,0.000000,8.461538
5,Olympic National Park,13,36.846154,23.307692,60.153846
6,Death Valley National Park,12,11.333333,52.083333,63.416667
7,Yellowstone National Park,12,179.083333,0.000000,179.083333
8,North Cascades National Park,11,26.636364,7.818182,34.454545
9,Joshua Tree National Park,9,47.000000,8.555556,55.555556


In [8]:
%%sql
SELECT
    p.name as park_name,
    COUNT(CASE WHEN c.numberofsitesreservable > 0 THEN 1 END) as num_reservable_campgrounds,
    COUNT(CASE WHEN c.numberofsitesfirstcomefirstserve > 0 THEN 1 END) as num_fcfs_campgrounds,
FROM nps_public_data.campgrounds c
INNER JOIN nps_public_data.parks p
    ON c.parkcode = p.parkcode
    AND p.designation = 'National Park'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5

,park_name,num_reservable_campgrounds,num_fcfs_campgrounds
0,Isle Royale,17,36
1,Guadalupe Mountains,13,0
2,Great Smoky Mountains,13,0
3,Yosemite,12,4
4,Yellowstone,12,0
